## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Mataura,NZ,-46.19,168.86,50.00,79,100,5.01,light rain
1,1,Torbay,CA,47.67,-52.73,61.57,36,20,10.36,few clouds
2,2,Codrington,AU,-38.27,141.97,40.30,67,5,8.23,clear sky
3,3,Kapaa,US,22.08,-159.32,79.00,73,1,8.05,clear sky
4,4,Dikson,RU,73.51,80.55,30.34,93,100,6.71,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.08,-159.32,79.00,73,1,8.05,clear sky
6,6,Sola,VU,-13.88,167.55,77.67,83,90,6.91,overcast clouds
13,13,Hilo,US,19.73,-155.09,86.29,74,40,5.75,scattered clouds
22,22,Butaritari,KI,3.07,172.79,82.87,77,100,14.70,overcast clouds
40,40,Rikitea,PF,-23.12,-134.97,75.07,73,100,12.55,light rain
46,46,Georgetown,MY,5.41,100.34,83.89,86,40,5.75,scattered clouds
49,49,Atuona,PF,-9.80,-139.03,79.77,62,1,7.76,clear sky
50,50,Akdepe,TM,42.06,59.38,89.60,13,4,8.14,clear sky
51,51,Adrar,MR,20.50,-10.07,92.43,25,100,12.62,overcast clouds
52,52,Hambantota,LK,6.12,81.12,80.94,89,40,19.17,scattered clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

city_id                214
City                   214
Country                214
Lat                    214
Lng                    214
Max Temp               214
Humidity               214
Cloudiness             214
Wind Speed             214
Current Description    214
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna()
clean_cities_df.head()

,city_id,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Kapaa,US,22.08,-159.32,79.00,73,1,8.05,clear sky
6,6,Sola,VU,-13.88,167.55,77.67,83,90,6.91,overcast clouds
13,13,Hilo,US,19.73,-155.09,86.29,74,40,5.75,scattered clouds
22,22,Butaritari,KI,3.07,172.79,82.87,77,100,14.70,overcast clouds
40,40,Rikitea,PF,-23.12,-134.97,75.07,73,100,12.55,light rain


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,79.00,clear sky,22.08,-159.32,
6,Sola,VU,77.67,overcast clouds,-13.88,167.55,
13,Hilo,US,86.29,scattered clouds,19.73,-155.09,
22,Butaritari,KI,82.87,overcast clouds,3.07,172.79,
40,Rikitea,PF,75.07,light rain,-23.12,-134.97,
46,Georgetown,MY,83.89,scattered clouds,5.41,100.34,
49,Atuona,PF,79.77,clear sky,-9.80,-139.03,
50,Akdepe,TM,89.60,clear sky,42.06,59.38,
51,Adrar,MR,92.43,overcast clouds,20.50,-10.07,
52,Hambantota,LK,80.94,scattered clouds,6.12,81.12,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        #print("Hotel found")
    except (IndexError):
            print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()

In [10]:
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Kapaa,US,79.00,clear sky,22.08,-159.32,Sheraton Kauai Coconut Beach Resort
6,Sola,VU,77.67,overcast clouds,-13.88,167.55,Ulkel Guest House
13,Hilo,US,86.29,scattered clouds,19.73,-155.09,Hilo Hawaiian Hotel
22,Butaritari,KI,82.87,overcast clouds,3.07,172.79,Isles Sunset Lodge
40,Rikitea,PF,75.07,light rain,-23.12,-134.97,Pension Maro'i
...,...,...,...,...,...,...,...
671,Labuan,MY,79.16,few clouds,5.28,115.24,Tiara Labuan Hotel
672,Champerico,GT,91.94,few clouds,14.30,-91.92,"Hotel y restaurante ""El Diamante"""
677,Satipo,PE,87.15,overcast clouds,-11.25,-74.64,Hotel Mayros
679,Buala,SB,78.80,light rain,-8.14,159.59,Maringe Lagoon Lodge


In [11]:
# 8a. Create the output File (CSV)
clean_hotel_df.to_csv("Weather_Database/WeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv("WeatherPy_vacation.csv", index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))